In [2]:
def load_words(filename):
    with open(filename, 'r') as file:
        words = file.read().splitlines()
    return words

words = load_words('../words_alpha.txt')

In [5]:
import re

def count_syllables(word):
    word = word.lower()
    # Find vowel groups
    syllable_count = len(re.findall(r'[aeiouy]+', word))
    # Adjust for silent 'e'
    if word.endswith('e'):
        syllable_count -= 1
    # Ensure at least 1 syllable
    return max(1, syllable_count)

# Apply the heuristic
syllable_data = [(word, count_syllables(word)) for word in words]

In [ ]:
import pyodbc
import os
from dotenv import load_dotenv

load_dotenv()
server = os.getenv('SERVER')
db = os.getenv('DATABASE')

# Connect to SQL Server database
conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    f'SERVER={server};'
    f'DATABASE={db};'
    'Trusted_Connection=yes;'
)
cursor = conn.cursor()